# Scraping de Metacritic
 L'objectif du scraping du site Metacritic est d'obtenir les informations ci-dessous :
 - Le nom du jeu-vidéo
 - la plateforme correspondant au jeu (sachant qu'un jeu peut être disponible sur plusieurs plateformes)
 - Le studio qui l'a développé
 - La note obtenue par le jeu auprès des utilisateurs
 - La note donnée par le site Metacritic

La première étape consiste à vérifier les repertoires qui sont autorisés au scraping en regardant le fichier : https://www.metacritic.com/robots.txt.
Les jeux-videos sur le site Metacritic sont rangés dans des listes déroulantes par plateformes (ex: Ps5, Ps4...). 
Dans chaque plateforme, nous avons des pages numérotées (Ex: page 1 à page 15) sur lesquelles nous avons les liens des jeux. 
Ces liens conduisent à une page de détails pour chaque jeu sur laquelle les informations recherchées sont disponibles.

Pour arriver à scraper toutes les données, nous allons procéder comme suit :

1. Charger les plateformes via https://www.metacritic.com/game (finalement manuellement en raison des différentes disparités sur les règles de nomenclature)
2. Récupérer tous les liens des jeux
3. Scraper les liens des jeux afin d'obtenir les informations détaillés
4. Consituter le dataset final


Pour en savoir plus sur les notes du site (Metascrore) : https://www.metacritic.com/about-metascores


In [1]:
# importer les librairies utilisées
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from urllib.request import ssl
import numpy as np

In [41]:
#Nombre des jeux-vidéos scrapés
count = 0

#urls non scrapés
url_plateforme_non_scrapes = []
url_page_non_scrapes = []
url_game_non_scrapes = []
url_games = []

ssl._create_default_https_context = ssl._create_unverified_context #utiliser une connexion ssl

def scrape_url(url):
    """
    Cette fonction a pour rôle de scraper la page fournie par url
    Nous utilisons l'agent Mozilla pour éviter des pare-feux 
    applicatifs qui peuvent bloquer les requêtes provenant directement de python
    """
    try:
        page = requests.get(url,headers={'User-Agent': 'Mozilla/5.0'})
        soup = BeautifulSoup(page.content, 'html.parser')
    except Exception as e:
        print("url : " + str(url))
        print("Error :" + str(e))
        soup = []
    return soup

def __getDetailsGame__(game_url):
    """
    Récupérer les données détaillées pour chaque jeu
    
    ******
    paramètres: 
    game_url : url pour accéder aux données détaillées du jeu
    ******
    """
    url_details_game = "https://www.metacritic.com"+str(game_url)
    
    #initialisation de soup de beautiful
    soup = []
    data = []
    
    #Scraping de la page transmise
    soup = scrape_url(url_details_game)
    
    
    if(len(soup)==0):
        print("Url inaccessible : " + str(url_details_game))
    else:
        metascore_html = soup.find(name = 'span', attrs = {'itemprop': 'ratingValue'})
        userscore_html_tbd = soup.find(name='div' , attrs={'class':'metascore_w user large game tbd'})
        userscore_html_mixed = soup.find(name='div' , attrs={'class':'metascore_w user large game mixed'})
        userscore_html_positive = soup.find(name='div' , attrs={'class':'metascore_w user large game positive'})
        userscore_html_negative = soup.find(name='div' , attrs={'class':'metascore_w user large game negative'})
        studio_html = soup.find(name = 'a', attrs = {'class': 'button'})
        date_html = soup.select('.release_data .data')
        game_name_html = soup.find(name='h1')
        plateforme_html = soup.select('.platform a')
        
        
        jeu = None if game_name_html == None else game_name_html.text.strip()
        metascore = None if metascore_html== None else metascore_html.text.strip()
        userscore_tdb = None if userscore_html_tbd== None else userscore_html_tbd.text.strip()
        userscore_mixed = None if userscore_html_mixed== None else userscore_html_mixed.text.strip()
        userscore_positive = None if userscore_html_positive== None else userscore_html_positive.text.strip()
        userscore_negative = None if userscore_html_negative== None else userscore_html_negative.text.strip()
        studio = None if studio_html== None else studio_html.text
        date = None if date_html == [] else date_html[0].text.strip()
        plateforme = None if plateforme_html == [] else plateforme_html[0].text.strip()
        
        
        data = {
            'jeu':jeu,
            'plateforme-url':plateforme,
            'date':date,
            'metascore': metascore,
            'userscore_tbd':userscore_tdb,
            'userscore_mixed':userscore_mixed,
            'userscore_positive':userscore_positive,
            'userscore_negative':userscore_negative,
            'studio':studio
        }
        
        return data
    

#Les plateformes
plateformes = ['ps4','ps5','xbox-series-x','xboxone','switch','pc','ios','stadia','ps3','ps2',
           'ps','xbox360','xbox','wii','ds','gamecube','n64','gba','psp','dreamcast','wii-u','3ds','vita']

#Fonction pour ajouter les links des jeux dans le tableau d'url à partir de la liste des plateformes
def ___GetGameLinkFromPlateformList(plateformes):
    """
    Cette fonction permet de récupérer les liens des jeux pour chaque plateforme.
    ***************
    Paramètres :
    - plateforme : liste des plateformes disponibles 
    ***************
    """
    count = 0
    # Log Heure de début
    print("Début : " + str(datetime.today()))

    #On parcours les plateformes pour trouver
    for p in plateformes:
        """
        Url qui liste les jeux par plateforme. 
        éSur cette partie nous pouvons avoir plusieurs page. 
        Nous recupérons le nombre max de page et faisons une boucle jusqu'a atteindre ce nombre
        """
        url_principal = "https://www.metacritic.com/browse/games/release-date/available/"+p+"/date"


        #scraper l'url principale
        soup = scrape_url(url_principal)


        #Si l'url scrapé renvoie une erreur, nous ne déroulons pas le process normal sinon nous le déroulons 
        if(len(soup)!=0):
            num_page = soup.select(".page_num")

            #Les numéros de page 
            #Si le numéro de page est valide, il ya plusieurs pages de liens de jeuxSinon il n'y a qu'une seule page à scraper
            if(num_page):

                #Nous retenons le numéro maximum de page
                val_range = int(num_page[-1].text)


                #Nous parcourons les pages à scraper pour obtenir tous les liens hypertexte des jeux
                for i in range(0,val_range):
                    #Url pour chaque page de liste de jeux par plateforme
                    url_page_game = "https://www.metacritic.com/browse/games/release-date/available/"+str(p)+"/date?page="+str(i)
                    
                    print("url de page numerotée : " + url_page_game)
                    #scraper l'url paginé
                    soup = scrape_url(url_page_game) 

                    #Si le scraping s'est bien passé
                    if(len(soup)!=0):
                        #Récupération du bloc contenant les liens hypertextes
                        bloc_list_game = soup.find_all('table', {'class':"clamp-list"})

                        #Parcourir les blocs de liens
                        for b in bloc_list_game:
                            #Récupérer les liens
                            for a in b.find_all('a', href=True,class_="title"): 
                                if a.text: 
                                    url_games.append(a['href'])
                                    count+=1
                                    print("Nombre de liens : " + str(count))
                    else:
                        url_page_non_scrapes.append(url_page_game)

            else:
                #Il y a une seule page à scraper
                url_page_game = "https://www.metacritic.com/browse/games/release-date/available/"+str(p)+"/date"

                #scraper l'url principale
                soup = scrape_url(url_page_game)
                
                print("url de page directe : " + url_page_game)

                #Si le scraping s'est bien passé
                if(len(soup)!=0):
                    #Récupération du bloc contenant les liens hypertextes
                        bloc_list_game = soup.find_all('table', {'class':"clamp-list"})

                        #Parcourir les blocs de liens
                        for b in bloc_list_game:
                            #Récupérer les liens
                            for a in b.find_all('a', href=True,class_="title"): 
                                if a.text: 
                                    url_games.append(a['href'])
                                    count+=1
                                    print("Nombre de liens : " + str(count))

                else:
                    url_page_non_scrapes.append(url_page_game)

        else:
            print("URL principale - scraping non valide")
            url_plateforme_non_scrapes.append(url_principal)

    # Log Heure de fin
    print("Fin : " + str(datetime.today()))
    return True




# Charger les liens des plateformes
___GetGameLinkFromPlateformList(plateformes)
print("Nombre de liens de jeux : " + str(len(url_games)))

# téléchargeer les liens dans un csv à preserver pour éviter de reprendre tout en cas de souci de connexion ou de plantage de jupyter notebook
df_links = pd.DataFrame(url_games,columns=['link'])

df_links.to_csv('../data/temp/metacritic/links.csv',sep=',')

# Récupérer les liens
df_links = pd.read_csv('../data/temp/metacritic/links.csv',sep=',',index_col='Unnamed: 0')

"""
Il y avait environ 117000 liens à scraper. Le notebook jupyter se plantait fréquemment. J'ai donc opté pour un découpage du 
dataframe des liens par lot de 5000. J'ai exécuté la portion de code suivante dans 24 notebook avec en entrée chaque lot de 5000
liens de jeux.

"""

In [ ]:
"""
Pour chaque lien dans notre liste url_game, ou notre dataFrame links, nous allons récupérer les infos détaillées
Ce bloc est à exécuter pour chaque lot de links ou en un bloc.

Exemple de découpage avant exécution
df_links = df_links[df_links.index<5000]

"""
#Tous les jeux-vidéos scrapés
allgames = []
count_games_data = 0

for l in df_links.link:
    #Récupérer les détails des jeux
    allgames.append(__getDetailsGame__(l))
    count_games_data+=1
    print("Nombre de data-jeu scrapés : " + str(count_games_data))

"""
Génération du dataset
"""

# Quel est le nombre total de ligne générées
print("Nombre total de lignes générées:" + str(len(allgames)))

#Supprimer les lignes "None"
allgames_save = [a for a in allgames if a is not None ]

#Création du dataset
dataset = pd.DataFrame(allgames_save)

# Remplacer les nan(None) par 0
dataset[
    ['metascore',
     'userscore_tbd',
     'userscore_mixed',
     'userscore_positive',
     'userscore_negative']
]= dataset[['metascore','userscore_tbd','userscore_mixed','userscore_positive','userscore_negative']].fillna('0')

# Remplacer les tdb par 0
dataset['userscore_tbd'] = dataset['userscore_tbd'].replace('tbd','0')

#Convertir les valeurs des notes en float
dataset[
    ['metascore',
     'userscore_tbd',
     'userscore_mixed',
     'userscore_positive',
     'userscore_negative']
] = dataset[
    ['metascore',
     'userscore_tbd',
     'userscore_mixed',
     'userscore_positive',
     'userscore_negative']
].astype('float')

#Trouver la note user en additionnant les différentes notes: l'une des notes seulement renseignée
dataset['userscore'] = dataset['userscore_tbd'] + dataset['userscore_mixed'] + dataset['userscore_positive'] + dataset['userscore_negative']

#Supprimer les notes intermédiaires
dataset = dataset.drop(['userscore_tbd','userscore_mixed','userscore_positive','userscore_negative'],axis=1)
dataset.to_csv('../data/temp/metacritic/MetacriticExample.csv',sep=',')

In [89]:
# Récupérer les datasets scrapés après découpage
df_games_1 = \
pd.read_csv('../data/temp/metacritic/MetacriticPart1.csv',sep=',',index_col='Unnamed: 0')

df_games_2 = \
pd.read_csv('../data/temp/metacritic/MetacriticPart2.csv',sep=',',index_col='Unnamed: 0')

df_games_3 = \
pd.read_csv('../data/temp/metacritic/MetacriticPart3.csv',sep=',',index_col='Unnamed: 0')

df_games_4 = \
pd.read_csv('../data/temp/metacritic/MetacriticPart4.csv',sep=',',index_col='Unnamed: 0')

df_games_5 = \
pd.read_csv('../data/temp/metacritic/MetacriticPart5.csv',sep=',',index_col='Unnamed: 0')

df_games_6 = \
pd.read_csv('../data/temp/metacritic/MetacriticPart6.csv',sep=',',index_col='Unnamed: 0')


df_games_7 = \
pd.read_csv('../data/temp/metacritic/MetacriticPart7.csv',sep=',',index_col='Unnamed: 0')

df_games_8 = \
pd.read_csv('../data/temp/metacritic/MetacriticPart8.csv',sep=',',index_col='Unnamed: 0')

df_games_9 = \
pd.read_csv('../data/temp/metacritic/MetacriticPart9.csv',sep=',',index_col='Unnamed: 0')

df_games_10 = \
pd.read_csv('../data/temp/metacritic/MetacriticPart10.csv',sep=',',index_col='Unnamed: 0')

df_games_11 = \
pd.read_csv('../data/temp/metacritic/MetacriticPart11.csv',sep=',',index_col='Unnamed: 0')

df_games_12 = \
pd.read_csv('../data/temp/metacritic/MetacriticPart12.csv',sep=',',index_col='Unnamed: 0')

df_games_13 = \
pd.read_csv('../data/temp/metacritic/MetacriticPart13.csv',sep=',',index_col='Unnamed: 0')

df_games_14 = \
pd.read_csv('../data/temp/metacritic/MetacriticPart14.csv',sep=',',index_col='Unnamed: 0')

df_games_15 = \
pd.read_csv('../data/temp/metacritic/MetacriticPart15.csv',sep=',',index_col='Unnamed: 0')

df_games_16 = \
pd.read_csv('../data/temp/metacritic/MetacriticPart16.csv',sep=',',index_col='Unnamed: 0')

df_games_17 = \
pd.read_csv('../data/temp/metacritic/MetacriticPart17.csv',sep=',',index_col='Unnamed: 0')

df_games_18 = \
pd.read_csv('../data/temp/metacritic/MetacriticPart18.csv',sep=',',index_col='Unnamed: 0')

df_games_19 = \
pd.read_csv('../data/temp/metacritic/MetacriticPart19.csv',sep=',',index_col='Unnamed: 0')

df_games_20 = \
pd.read_csv('../data/temp/metacritic/MetacriticPart20.csv',sep=',',index_col='Unnamed: 0')

df_games_21 = \
pd.read_csv('../data/temp/metacritic/MetacriticPart21.csv',sep=',',index_col='Unnamed: 0')
#Quelques nan au niveau des plateformes içi

df_games_22 = \
pd.read_csv('../data/temp/metacritic/MetacriticPart22.csv',sep=',',index_col='Unnamed: 0')

df_games_23 = \
pd.read_csv('../data/temp/metacritic/MetacriticPart23.csv',sep=',',index_col='Unnamed: 0')

df_games_24 = \
pd.read_csv('../data/temp/metacritic/MetacriticPart24.csv',sep=',',index_col='Unnamed: 0')


df_games = pd.concat([
    df_games_1,df_games_2,df_games_3,df_games_4,df_games_5,df_games_6,df_games_7,df_games_8,df_games_9,df_games_10,
    df_games_11,df_games_12,df_games_13,df_games_14,df_games_15,df_games_16,df_games_17,df_games_18,df_games_19,
    df_games_20,df_games_21,df_games_22,df_games_23,df_games_24
])

df_games.to_csv('../data/temp/metacritic/AllPartsMetacritic.csv',sep=',')

In [90]:
#Renommage des colonnes du dataframe 
df_games = df_games.rename(
    columns={
        'plateforme-url':'Plateforme',
        'jeu':'Name','date':'Date',
        'studio':'Studio','metascore':'Metascore',
        'userscore':'Userscore'})

In [91]:
"""
Supprimer les lignes avec les colonnes suivantes à nan (Jeu,Date, Plateform,studio)
"""
# Données non correctes
df_games = df_games.dropna(axis = 0, how = 'all', subset = ['Date','Plateforme','Studio'])

In [92]:
"""
Convertir les dates
"""

#Supprimer les nan dans la date 
df_games = df_games.dropna(axis = 0, how = 'all', subset = ['Date'])
years = []
for d in df_games['Date']:
    s = d[-5:]
    years.append(s)

df_games['Year'] = years

# Supprimer les lignes avec les dates incorrectes 
df_games = df_games[(df_games['Year']!='TBA') & (df_games['Year']!='ccess')]

#Mettre la date de sortie au format MM/AAAA
def __ManageDate(d):
    if 'TBA' in str(d):
        d = d.strip('TBA')
        date = 'Jan 01,' + str(d)
    elif 'October 2009' in d:
        d = d.strip('October')
        date = 'Oct 01,' + str(d)
    elif len(d.strip())==4:
        date = 'Jan 01, ' + str(d)
    else:
        date = d
    return datetime.strptime(date, '%b %d, %Y')
    
df_games['Date'] = df_games['Date'].apply(lambda x : __ManageDate(x))

"""
Mettre à jour les types 
"""
df_games['Year'] = df_games['Year'].astype(int)
df_games[['Metascore','Userscore']] = df_games[['Metascore','Userscore']].astype(float)
#Mettre les notes Metacritic à l'échelle 
df_games['Metascore'] = df_games['Metascore'].apply(lambda x : x/10)

In [93]:
#Supprimer les doublons
df_games = df_games.drop_duplicates(keep='last')

In [94]:
"""
Restreindre notre dataset aux jeux et plateformes présentes dans le dataset initial
"""
#Plateforme non disponible dans le dataset initial
plateformeNotExist = ['PlayStation 5','Xbox Series X','Switch','iOS','Stadia'] 

#Filtrage des données pour les plateformes existentes dans le dataset initial
df_games = df_games[~df_games['Plateforme'].isin(plateformeNotExist)]
#96978 

In [95]:
"""
Table de correspondance des plateformes.
Cette table sert à joindre les dénomminations des plateformes du dataset Metacritic et du dataset inital
"""
cor_Plateform = [
    {
        'metacritic':'Wii',
        'source':'Wii'
    },
    
     {
        'metacritic':'Xbox 360',
        'source':'X360'
    },
    {
        'metacritic':'PlayStation 3',
        'source':'PS3'
    },
    {
        'metacritic':'PlayStation 2',
        'source':'PS2'
    },
    {
        'metacritic':'Game Boy Advance',
        'source':'GBA'
    },
    {
        'metacritic':'PlayStation 4',
        'source':'PS4'
    },
    {
        'metacritic':'Nintendo 64',
        'source':'N64'
    },
    {
        'metacritic':'PlayStation',
        'source':'PS'
    },
     {
        'metacritic':'PC',
        'source':'PC'
    },
    {
        'metacritic':'PSP',
        'source':'PSP'
    },
     {
        'metacritic':'Xbox One',
        'source':'XOne'
    },
     {
        'metacritic':'GameCube',
        'source':'GC'
    },
     {
        'metacritic':'Wii U',
        'source':'WiiU'
    },
    {
        'metacritic':'Dreamcast',
        'source':'DC'
    },
     {
        'metacritic':'PlayStation Vita',
        'source':'PSV'
     },
    {
        'metacritic':'DS',
        'source':'DS'
     },
    {
        'metacritic':'3DS',
        'source':'3DS'
     },
    {
        'metacritic':'Xbox',
        'source':'XB'
     }
    
]

df_cor = pd.DataFrame(cor_Plateform)

In [96]:
"""
Merger le dataset Metacritic avec la correspondances des plateformes
"""
df_games = df_games.merge(df_cor,left_on='Plateforme',right_on='metacritic', how='left').drop_duplicates()
df_games = df_games.drop(columns=['metacritic'])
df_games = df_games.rename(columns={'source':'CodePlateforme'})

In [97]:
"""
Préparation à la jointure avec les autres datasets
"""

newGamesNames = []
for d in df_games['Name']:
    newname = ''.join(e for e in d if e.isalnum())
    newGamesNames.append(newname.lower())

df_games['NameFormated'] = newGamesNames
df_games['join'] = df_games['CodePlateforme'] + df_games['NameFormated']
df_games = df_games.drop(columns=['Plateforme','Name','Year','CodePlateforme','CodePlateforme'])

In [98]:
df_games.to_csv('../data/01_GameSpy_Scraping_Metacritic.csv',sep=',')